In [10]:
# Import libraries
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2

from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", tf.test.gpu_device_name())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

Num GPUs Available: 1
TensorFlow is using: /device:GPU:0
GPU memory growth enabled.


I0000 00:00:1743330933.171364    2088 gpu_device.cc:2022] Created device /device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


In [11]:
def load_data(spectrogram_dir, label_dir):
    spectrograms = []
    labels = []

    # Load spectrograms
    spectrogram_files = sorted(os.listdir(spectrogram_dir))
    label_files = sorted(os.listdir(label_dir))

    total_files = len(spectrogram_files)

    for spec_file, label_file in tqdm(zip(spectrogram_files, label_files), 
                                        total=total_files, 
                                        desc="Loading Data", 
                                        unit="file"):
        # Load data
        spec_data = np.load(os.path.join(spectrogram_dir, spec_file))
        label_data = np.load(os.path.join(label_dir, label_file)).T

        # Append if there is valid data
        if len(spec_data) > 0:
            spectrograms.append(spec_data)
            labels.append(label_data)

    # Convert to numpy arrays 
    if spectrograms:
        X = np.vstack(spectrograms)
        Y = np.vstack(labels)
        return X, Y
    else:
        return np.array([]), np.array([])

In [12]:
X_train, Y_train = load_data("spectrograms_train", "labels_train")
X_train = np.expand_dims(X_train, axis=-1)
print("Final X_train:", X_train.shape, "Final Y_train:", Y_train.shape)

Loading Data: 100%|██████████| 1289/1289 [00:01<00:00, 807.67file/s]


Final X_train: (319508, 64, 42, 1) Final Y_train: (319508, 8)


In [13]:
# Occurrences
class_counts = np.sum(Y_train, axis=0)
total_samples = len(Y_train)

for i, count in enumerate(class_counts):
    print(f"Class {i}: {count} occurrences ({count / total_samples:.2%} of the data)")

Class 0: 128695 occurrences (40.28% of the data)
Class 1: 13526 occurrences (4.23% of the data)
Class 2: 27686 occurrences (8.67% of the data)
Class 3: 125932 occurrences (39.41% of the data)
Class 4: 171444 occurrences (53.66% of the data)
Class 5: 257671 occurrences (80.65% of the data)
Class 6: 155212 occurrences (48.58% of the data)
Class 7: 271620 occurrences (85.01% of the data)


In [14]:
model = Sequential([
    Input(shape=(64, 42, 1)), 
    Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),
    Dropout(0.3),  
    Dense(8, activation='sigmoid', dtype='float32')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 64, 42, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 21, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       163,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,720 (733.28 KB)

 Trainable params: 187,720 (733.28 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train model
history = model.fit(X_train, Y_train, batch_size=64, epochs=10, verbose=1)

# Final loss & accuracy
final_loss = history.history['loss'][-1]
final_accuracy = history.history['binary_accuracy'][-1]
print(f"Final Loss: {final_loss:.4f}, Final Accuracy: {final_accuracy:.4f}")

2025-03-30 03:36:08.558459: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3435350016 exceeds 10% of free system memory.


Epoch 1/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - binary_accuracy: 0.8120 - loss: 0.4049
Epoch 2/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - binary_accuracy: 0.8783 - loss: 0.2993
Epoch 3/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.8898 - loss: 0.2813
Epoch 4/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - binary_accuracy: 0.8936 - loss: 0.2750
Epoch 5/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.8958 - loss: 0.2718
Epoch 6/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.8981 - loss: 0.2678
Epoch 7/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - binary_accuracy: 0.8991 - loss: 0.2661
Epoch 8/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.9003 - loss: 0.2640
Epoch 9/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - binary_accuracy: 0.9005 - loss: 0.2636
Epoch 10/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.9009 - loss: 0.2626
Final Loss: 0.2634, Final Accuracy: 0.9

In [16]:
# Get predictions
Y_pred = model.predict(X_train)

# Convert to binary (threshold the probabilities)
threshold = 0.5
Y_pred_binary = (Y_pred > threshold).astype(int)

# Calculate metrics
precision = precision_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
recall = recall_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
f1 = f1_score(Y_train, Y_pred_binary, average='samples', zero_division=0)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

9985/9985 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step
Precision: 0.8958
Recall: 0.8580
F1-Score: 0.8658


In [17]:
X_test, Y_test = load_data("spectrograms_test", "labels_test")
X_test = np.expand_dims(X_test, axis=-1)
Y_pred_test = model.predict(X_test)

Loading Data: 100%|██████████| 151/151 [00:00<00:00, 871.16file/s]


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [18]:
# --- Threshold Adjustment ---
num_classes = Y_test.shape[1]
thresholds = np.array([0.5] * num_classes)  # Default 0.5
# Adjust thresholds as needed (example):
thresholds[0] = 0.5
thresholds[1] = 0.3
thresholds[2] = 0.3
thresholds[3] = 0.3
thresholds[4] = 0.6
thresholds[6] = 0.4

Y_pred_test_binary = (Y_pred_test >= thresholds).astype(int)
# --- End Threshold Adjustment ---

precision_test = precision_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
recall_test = recall_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
f1_test = f1_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)

print(f"Test Precision: {precision_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")

# Compute per-class precision, recall, and F1-score
precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
    Y_test, Y_pred_test_binary, average=None, zero_division=0
)

# Occurrences
class_counts = np.sum(Y_train, axis=0) # Make sure Y_train is defined.
total_samples = len(Y_train)

for i in range(num_classes):
    print(f"Class {i} | Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, "
          f"F1={f1_per_class[i]:.4f} | Thresh={thresholds[i]:.1f}, Occurrences={int(class_counts[i])} ({class_counts[i] / total_samples:.2%})")

Test Precision: 0.8716
Test Recall: 0.8357
Test F1-Score: 0.8397
Class 0 | Precision=0.8069, Recall=0.6790, F1=0.7375 | Thresh=0.5, Occurrences=128695 (40.28%)
Class 1 | Precision=0.9490, Recall=0.6863, F1=0.7966 | Thresh=0.3, Occurrences=13526 (4.23%)
Class 2 | Precision=0.8066, Recall=0.7215, F1=0.7617 | Thresh=0.3, Occurrences=27686 (8.67%)
Class 3 | Precision=0.7221, Recall=0.8769, F1=0.7920 | Thresh=0.3, Occurrences=125932 (39.41%)
Class 4 | Precision=0.9131, Recall=0.6855, F1=0.7831 | Thresh=0.6, Occurrences=171444 (53.66%)
Class 5 | Precision=0.9492, Recall=0.9799, F1=0.9643 | Thresh=0.5, Occurrences=257671 (80.65%)
Class 6 | Precision=0.9537, Recall=0.8021, F1=0.8714 | Thresh=0.4, Occurrences=155212 (48.58%)
Class 7 | Precision=0.9616, Recall=0.9853, F1=0.9733 | Thresh=0.5, Occurrences=271620 (85.01%)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 42, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 21, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 5, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5120)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       327,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,054,682 (4.02 MB)

 Trainable params: 351,560 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 703,122 (2.68 MB)

In [11]:
model.save("instrument_classifier.h5")
model = tf.keras.models.load_model("instrument_classifier.h5", compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("instrument_classifier.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7e4d6wmg/assets


INFO:tensorflow:Assets written to: /tmp/tmp7e4d6wmg/assets


Saved artifact at '/tmp/tmp7e4d6wmg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 42, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  139966833294352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139969607725152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820442080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820447008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139969591649808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820549552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820563280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820561344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820464272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139966820451072: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743319977.726942     579 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743319977.726993     579 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-30 00:32:57.728157: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp7e4d6wmg
2025-03-30 00:32:57.728742: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-30 00:32:57.728753: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp7e4d6wmg
I0000 00:00:1743319977.735491     579 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-30 00:32:57.736669: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-30 00:32:57.769411: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp7e4d6wmg
2025-03-30 00:32:57.778421: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 